In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:80% !important;}
</style>
"""))

# 연관 분석

## 연관 분석 개요
- 데이터들 사이에서 자주 발생하는 속성을 찾고 그 속성들 사이에 어느 정도 있는지를 분석하는 방법
- 활용분야: 상품진열, 사기보험적발, 카탈로그디자인, 신상품 카테고리 구성
- 연관성분석 관련 지표
    - 지지도(조건결과항목수/전체수)
    - 신뢰도(조건결과항목수/조건항목수)
    - 향상도(우연히 발생한 규칙인지 여부) = 조건결과지도/(조건지지도 * 결과지지도)
    - 1: 상관관계 없음, >1: 양의 상관관계, <1: 음의 상관관계
- [조건] > [결과]:  지지도  신뢰도  향상도

In [2]:
import csv
with open('cf_basket.csv', 'r', encoding='utf-8') as cf:
    csvdata = csv.reader(cf)
    transaction = list(csvdata)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['콜라', '맥주', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인', '콜라']]

## 연관분석
- pip install apyori

### 연관 규칙 생성

In [11]:
from apyori import apriori
rules = apriori(transaction, min_support=0.2, min_confidence=0.1)
list(rules)[10] #lift 분석

RelationRecord(items=frozenset({'소주', '콜라'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주', '콜라'}), confidence=0.6, lift=1.0), OrderedStatistic(items_base=frozenset({'소주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'소주'}), confidence=0.6, lift=1.0)])

In [14]:
print('조건  >  결과\t지지도\t신뢰도\t향상도')
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join(x for x in item[0])
        #lhs = [x for x in item[0]]
        rhs = ', '.join(x for x in item[1])
        confidence = item[2]
        lift = item[3]

조건  >  결과	지지도	신뢰도	향상도


## 뉴스기사 연관 분석 실습

### 뉴스 RSS 이용 연관분석 실습

In [23]:
import requests
from bs4 import BeautifulSoup
rss_url = 'https://rss.joins.com/joins_money_list.xml'
money_response = requests.get(rss_url)
money_soup = BeautifulSoup(money_response.content, "xml")
link_list = money_soup.select("item > link")
#link_list = [l.text for l in link_list]

In [24]:
from konlpy.tag import Kkma
kkma = Kkma()
news = []
for link in link_list:
    news_response = requests.get(link.text)
    news_soup = BeautifulSoup(news_response.content, "html.parser")
    content = news_soup.select_one("div#article_body").text
    nouns = list(filter(lambda word: len(word)>1, kkma.nouns(content)))#한 글자 이상
    news.append(nouns)
print(news)

[['26', '26일', '승진', '대상', '대상그룹', '그룹', '부회장', '회장', '사진', '44', '전무', '지주', '지주회사', '회사', '대상홀딩스와', '홀딩', '스와', '임창', '명예', '명예회장', '장녀', '차녀', '임상', '임상민', '41', '자매', '경영', '체제', '본격', '궤도', '해석', '부회', '대상홀딩스', '딩스', '전략', '전략담당', '담당', '중역', '기존', '마케팅', '마케팅담당', '보직', '동시', '수행', '관련', '대상홀딩', '이날', '정기', '주주', '주주총회', '총회', '사내', '사내이사', '이사', '신규', '선임', '안건', '의결', '연세대', '경영학', '공부', '이후', '미국', '뉴욕', '뉴욕대', '심리학', '전공', '1998', '1998년', '이재', '삼성', '삼성전자', '전자', '결혼', '2009', '2009년', '이혼', '2012', '2012년', '크리', '티브', '디렉터', '직책', '합류', '2014', '2014년', '청정원', '정원', '대규모', '리뉴얼', '주도', '주도해', '브랜드', '이미지', '강화', '2016', '2016년', '안주야', '주야', '출시', '국내', '안주', '가정', '가정간편식', '편식', '시장', '개척', '설명', '관계자', '변화', '정확', '정보', '습득', '실행', '차원', '중장기', '방향', '일관', '추진', '적임자', '판단', '업계', '지주사인', '사인', '대상홀딩스에', '언니', '주력', '사업회', '사업회사인', '동생', '본격화', '전체', '투자', '부문', '실질적', '식품', '식자재', '자재', '유통', '사업', '지난해', '상민', '일각', '후계', '후계구도', '구도', '관측', '당초', '지분', '승계', '중심', '가업

In [25]:
from apyori import apriori
rules = apriori(news, min_support=0.3, min_confidence=0.2)
result = list(rules)
for r in result:
    print(r)

RelationRecord(items=frozenset({'10'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'19'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'19'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'26'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'26'}), confidence=0.6, lift=1.0)])
RelationRecord(items=frozenset({'26일'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'26일'}), confidence=0.6, lift=1.0)])
RelationRecord(items=frozenset({'결과'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'결과'}), confidence=0.3, lift=1.0)])
RelationRecord(items=frozenset({'계획'}), support=0.36666666666666664, ordered_statis

In [29]:
import pandas as pd
result_df = pd.DataFrame(None, columns=['lhs', 'rhs', 'support', 'confidence', 'lift'])
index = 0
for row in result:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ','.join(x for x in item[0])
        rhs = ','.join(x.strip() for x in item[1])
        confidence = item[2]
        lift = item[3]
        if lift != 1:
            result_df.loc[index] = [lhs, rhs, support, confidence, lift]
            index += 1
result_df

,lhs,rhs,support,confidence,lift
0,19,기자,0.300000,0.900000,1.173913
1,기자,19,0.300000,0.391304,1.173913
2,26,26일,0.600000,1.000000,1.666667
3,26일,26,0.600000,1.000000,1.666667
4,26,관련,0.433333,0.722222,0.984848
...,...,...,...,...,...
729,"이사,주주총회,그룹,주주",총회,0.300000,1.000000,3.333333
730,"이사,총회,그룹,주주",주주총회,0.300000,1.000000,3.333333
731,"이사,주주총회,총회,그룹",주주,0.300000,1.000000,3.000000
732,"주주,주주총회,총회,그룹",이사,0.300000,1.000000,3.333333


In [31]:
pd.options.display.max_rows = 732
result_df

,lhs,rhs,support,confidence,lift
0,19,기자,0.300000,0.900000,1.173913
1,기자,19,0.300000,0.391304,1.173913
2,26,26일,0.600000,1.000000,1.666667
3,26일,26,0.600000,1.000000,1.666667
4,26,관련,0.433333,0.722222,0.984848
...,...,...,...,...,...
729,"이사,주주총회,그룹,주주",총회,0.300000,1.000000,3.333333
730,"이사,총회,그룹,주주",주주총회,0.300000,1.000000,3.333333
731,"이사,주주총회,총회,그룹",주주,0.300000,1.000000,3.000000
732,"주주,주주총회,총회,그룹",이사,0.300000,1.000000,3.333333


In [32]:
result_df.loc[(result_df.lhs.str.contains("기자"))].sort_values(by=['lift'], ascending=False)

,lhs,rhs,support,confidence,lift
663,"관련,26,기자","26일,기사",0.333333,0.769231,2.307692
664,"26,기사,기자","26일,관련",0.333333,1.000000,2.307692
666,"26일,관련,기자","기사,26",0.333333,0.769231,2.307692
667,"26일,기사,기자","관련,26",0.333333,1.000000,2.307692
527,"기사,기자","관련,26",0.333333,0.833333,1.923077
555,"기사,기자","26일,관련",0.333333,0.833333,1.923077
658,"기사,기자","26일,26,관련",0.333333,0.833333,1.923077
697,"26일,이날,기자","관련,26",0.300000,0.818182,1.888112
695,"26일,관련,기자","26,이날",0.300000,0.692308,1.888112
694,"26,이날,기자","26일,관련",0.300000,0.818182,1.888112
